In [1]:
import sklearn
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df = pd.get_dummies(df, columns=['penalty'])
y_train = df.loc[:, 'time']
X_train = df.loc[:, ['penalty_l2', 'penalty_none', 'penalty_elasticnet', 'max_iter', 'n_samples', 'n_features', 'n_jobs', 'n_classes', 'flip_y']]

X_train['job_inv'] = 1 / X_train['n_jobs']
X_train['workload'] = X_train['max_iter'] * X_train['n_classes']  * X_train['n_samples'] * X_train['n_features'] * X_train['job_inv']
X_train['class_job'] = X_train['n_classes'] * X_train['job_inv']
X_train['samples_job'] = X_train['n_samples'] * X_train['job_inv']
X_train['iter_job'] = X_train['max_iter'] * X_train['job_inv']
X_train['feature_job'] = X_train['n_features'] * X_train['job_inv']

test_df = pd.read_csv('test.csv')

test_df = pd.get_dummies(test_df, columns=['penalty'])
X_test = test_df.loc[:, ['penalty_l2', 'penalty_none', 'penalty_elasticnet', 'max_iter', 'n_samples', 'n_features', 'n_jobs', 'n_classes', 'flip_y']]

X_test['job_inv'] = 1 / X_test['n_jobs']
X_test = X_test.drop(['n_jobs'],axis=1)
X_test['workload'] = X_test['max_iter'] * X_test['n_classes']  * X_test['n_samples'] * X_test['n_features'] * X_test['job_inv']
X_test['class_job'] = X_test['n_classes'] * X_test['job_inv']
X_test['samples_job'] = X_test['n_samples'] * X_test['job_inv']
X_test['iter_job'] = X_test['max_iter'] * X_test['job_inv']
X_test['feature_job'] = X_test['n_features'] * X_test['job_inv']


In [3]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=1000, max_features=0.5, subsample=0.5, learning_rate=0.01, max_depth=3, random_state=0, loss='ls').fit(X_train, y_train)

log_pred = model.predict(X_test)
log_pred = [score if score > 0 else 0 for score in log_pred]
pred = np.exp(log_pred)
test_result_df = pd.DataFrame(data={'ID': range(len(pred)), 'time': pred})

test_result_df.to_csv('test_result.csv', index=False)

ValueError: Number of features of the model must match the input. Model n_features is 15 and input n_features is 14 